In [1]:
from cryocat import cryomap  # For handling cryo-ET .mrc file formats
from pathlib import Path  # For file path manipulations
from PIL import Image  # For image processing
import imageio  # For saving image files

# Read the MRC file using cryomap
mrc = cryomap.read('tutorial_data/experimental_TS/0248_dose-filt_ali_bin8.mrc')

# Normalize the MRC data to 0–255 range and convert to uint8 format
mrc = ((mrc - mrc.min()) * (1 / (mrc.max() - mrc.min()) * 255)).astype('uint8')

# Define the path to save generated PNG files
PATH_ORIG = f'tutorial_data/orig'
Path(PATH_ORIG).mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist

# Iterate through slices along the Z-axis of the MRC volume and save as PNGs
for i in range(0, mrc.shape[2]):  # Loop through the third dimension (Z)
    image = Image.fromarray(mrc[:, :, i])  # Extract 2D slice
    if image.mode != 'RGB':  # Ensure image is in RGB format
        image = image.convert('RGB')
    imageio.imwrite(f'{PATH_ORIG}/{(i * 10):04}.png', image)  # Save as PNG with incremented name


In [ ]:
import subprocess as sp  # For running external commands
import shutil  # For file operations
import os  # For OS-level operations

# Define path to original PNGs
PATH_ORIG = 'tutorial_data/orig'

# Define base path for storing interpolated results
PATH_BASE = 'tutorial_data/tilts_cryoTIGER_model'

# Run interpolation using the cryoTIGER model
process = sp.Popen(f'python3 -m eval.interpolator_cli --pattern "{PATH_ORIG}" \
                    --model_path models/cryoTIGER_model \
                    --times_to_interpolate 1', shell=True)
process.wait()  # Wait for process to complete

# Move interpolated frames generated by the cryoTIGER model to a specific folder
if os.path.exists("tutorial_data/interpolated_frames_cryoTIGER_model"):
    shutil.rmtree("tutorial_data/interpolated_frames_cryoTIGER_model")  # Remove existing folder if it exists
shutil.move(f"{PATH_ORIG}/interpolated_frames", "tutorial_data/interpolated_frames_cryoTIGER_model")

# Run interpolation using the pre-trained Vimeo-90K model
process = sp.Popen(f'python3 -m eval.interpolator_cli --pattern "{PATH_ORIG}" \
                    --model_path models/pre-trained_Vimeo-90K_data_model \
                    --times_to_interpolate 1', shell=True)
process.wait()  # Wait for process to complete

# Move interpolated frames generated by the Vimeo-90K model to a specific folder
if os.path.exists("tutorial_data/interpolated_frames_pre-trained_model"):
    shutil.rmtree("tutorial_data/interpolated_frames_pre-trained_model")  # Remove existing folder if it exists
shutil.move(f"{PATH_ORIG}/interpolated_frames", "tutorial_data/interpolated_frames_pre-trained_model")


In [3]:
from cryocat import cryomap  # For handling cryo-ET .mrc file formats
from PIL import Image  # For image processing
from os import listdir  # For directory listing
import imageio.v3 as iio  # For reading images
import numpy as np  # For numerical operations

# Function to process interpolated frames and save them as an MRC file
def process_interpolated_frames(input_path, output_filename):
    
    tomo3d = []  # Initialize list to store 2D slices
    for tomo in sorted(listdir(input_path)):  # Process each file in sorted order
        img = Image.open(f'{input_path}/{tomo}').convert('L')  # Convert image to grayscale
        img.save(f'{input_path}/{tomo}')  # Save the converted image (overwrite)
        
        im = iio.imread(f'{input_path}/{tomo}')  # Read image data
        tomo3d.append(im)  # Append slice to list
    
    # Stack 2D slices along the Z-axis to reconstruct a 3D volume
    tomo3d = np.stack(tomo3d, axis=2)
    
    # Save the reconstructed 3D volume as an MRC file
    cryomap.write(tomo3d, output_filename, data_type=np.single)

# Process interpolated frames for the cryoTIGER model
PATH_GEN = f'tutorial_data/interpolated_frames_cryoTIGER_model/'
process_interpolated_frames(PATH_GEN, f"tutorial_data/interpolated_TS/demo_tilt_series_bin8_interpolated_cryoTIGER_model.mrc")

# Process interpolated frames for the pre-trained Vimeo-90K model
PATH_GEN = f'tutorial_data/interpolated_frames_pre-trained_model/'
process_interpolated_frames(PATH_GEN, f"tutorial_data/interpolated_TS/demo_tilt_series_bin8_interpolated_pre-trained_model.mrc")
